### Purpose: Predicting which plant a user has at home

### Use: The goal is to collect data and create a model that can effectively predict which plant the user has when uploading a photo on the platform.

### Problem type: Image Classification

### Data Collection: The data was collected by members of the team and was limited to the available plants in their household and surroundings. So some of the plants are also limited to one stage of its growing, for example the tomato has only pictures of its fruit unripened, so a plant with a red tomato could be misclassified.

### Design Choices: The choice was to use ML.NET due to the easier integration and experience that the team had using Net.Core, and also due to it's easier integration with Azure Functions. Due to this choice, some of the options were limited when building the model, after some research on the available libraries in C# for machine learning, SciSharp.TensorFlow and Sci.Keras were taken into account to build the neural networks but none of them allowed for using a pre-trained model as a base to a customized CNN, due to this reason, there was a limitation to use one of the available DNN pre-trained models in ML.NET which on the other hand only allows pre-trained models, in our case and due to the short amount of data, it was a better choice to use pre-trained models instead of only the CNN without the pre-trained model as a base.

In [2]:
// ML.NET Nuget packages installation
#r "nuget:Microsoft.ML,1.7.0"
#r "nuget:Microsoft.ML.Vision,1.7.0"    
#r "nuget:Microsoft.ML.ImageAnalytics,1.7.0"    
#r "nuget:Microsoft.Extensions.ML,1.7.0"
#r "nuget:SciSharp.TensorFlow.Redist-Windows-GPU,1.14.0"
//Install XPlot package
#r "nuget:XPlot.Plotly,2.0.0"

Installed Packages Microsoft.Extensions.ML, 1.7.0 Microsoft.ML, 1.7.0 Microsoft.ML.ImageAnalytics, 1.7.0 Microsoft.ML.Vision, 1.7.0 SciSharp.TensorFlow.Redist-Windows-GPU, 1.14.0 XPlot.Plotly, 2.0.0

In [20]:
using Microsoft.ML.Data;
using Microsoft.ML;
using Microsoft.ML.Vision;
using System.IO;
using System;
using static Microsoft.ML.Transforms.ValueToKeyMappingEstimator;
using System.Linq;

public static IEnumerable<ImageData> LoadImages(
            string folder,
            bool useFolderNameAsLabel = true)
            => LoadImagesFromDirectory(folder, useFolderNameAsLabel)
                .Select(x => new ImageData(x.imagePath, x.label));

public static IEnumerable<(string imagePath, string label)> LoadImagesFromDirectory(
            string folder,
            bool useFolderNameasLabel)
        {
            var imagesPath = Directory
                .GetFiles(folder, "*", searchOption: SearchOption.AllDirectories)
                .Where(x => Path.GetExtension(x) == ".jpg" || Path.GetExtension(x) == ".png");

            return useFolderNameasLabel
                ? imagesPath.Select(imagePath => (imagePath, Directory.GetParent(imagePath).Name))
                : imagesPath.Select(imagePath =>
                {
                    var label = Path.GetFileName(imagePath);
                    for (var index = 0; index < label.Length; index++)
                    {
                        if (!char.IsLetter(label[index]))
                        {
                            label = label.Substring(0, index);
                            break;
                        }
                    }
                    return (imagePath, label);
  });
}

        public static IEnumerable<ImagePredictionInput> LoadLocalImages(
            string folder,
            bool useFolderNameAsLabel = true)
        {
            return LoadImagesFromDirectory(folder, useFolderNameAsLabel)
                .Select(x => new ImagePredictionInput(
                    image: File.ReadAllBytes(x.imagePath),
                    label: x.label,
                    imageFileName: Path.GetFileName(x.imagePath)));
        }

public class ImagePredictionOutput
    {
        [ColumnName("Score")]
        public float[] Score;

        [ColumnName("PredictedLabel")]
        public string PredictedLabel;
    }

 public class ImageData
{
    public ImageData(string imagePath, string label)
    {
        ImagePath = imagePath;
        Label = label;
    }

    public readonly string ImagePath;
    public readonly string Label;
}

  public class ImagePredictionInput
    {
        public ImagePredictionInput()
        {
        }

        public ImagePredictionInput(byte[] image, string label, string imageFileName)
        {
            Image = image;
            Label = label;
            ImageFileName = imageFileName;
        }

        public byte[] Image {get;set;}

        public string Label {get;set;}

        public string ImageFileName {get;set;}
    }

private static void TrySinglePrediction(string imagesFolderPathForPredictions, MLContext mlContext, ITransformer trainedModel)
{
    var predictionEngine = mlContext.Model
        .CreatePredictionEngine<ImagePredictionInput, ImagePredictionOutput>(trainedModel);
    var testImages = LoadLocalImages(
            imagesFolderPathForPredictions, false);
    var imageToPredict = testImages.First();
    var prediction = predictionEngine.Predict(imageToPredict);

    Console.WriteLine(
                $"Image Filename : [{imageToPredict.ImageFileName}], " +
                $"Scores : [{string.Join(",", prediction.Score.OfType<float>().ToList().Max())}], " +
                $"Predicted Label : {prediction.PredictedLabel}");
}

const string initialDataset = @"C:\Users\luisf\Downloads\Machine Learning personal dataset\Machine Learning personal dataset";
const string initialReducedDataset =  @"C:\Users\luisf\Downloads\Machine Learning personal dataset - Cópia\Machine Learning personal dataset";
const string augmentedDataset = @"C:\Users\luisf\Downloads\Machine Learning personal dataset AUG";
const string finalDataset = @"C:\Users\luisf\Downloads\Machine Learning personal Final\Machine Learning personal dataset";

var mlContext = new MLContext(seed: 1);


(57,23): warning CS0649: Field 'ImagePredictionOutput.PredictedLabel' is never assigned to, and will always have its default value null

(54,24): warning CS0649: Field 'ImagePredictionOutput.Score' is never assigned to, and will always have its default value null



----

## First, the initial dataset was tested to understand what prediction could be retrieved from it using the pre-trained model ResNetv50

In [4]:
// Load the initial dataset and shuffle so it'll be better balanced
IEnumerable<ImageData> images = LoadImages(folder: initialDataset, useFolderNameAsLabel: true);
IDataView fullImagesDataset = mlContext.Data.LoadFromEnumerable(images);
IDataView shuffledFullImageFilePathsDataset = mlContext.Data.ShuffleRows(fullImagesDataset);

// Load Images with in-memory type within the IDataView and Transform Labels to Keys
IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.
    MapValueToKey(outputColumnName: "LabelAsKey", inputColumnName: "Label", keyOrdinality: KeyOrdinality.ByValue)
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: initialDataset,
        inputColumnName: "ImagePath"))
    .Fit(shuffledFullImageFilePathsDataset)
    .Transform(shuffledFullImageFilePathsDataset);

 // Split the data 80:20 into train and test sets, train and evaluate.
var trainTestData = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset, testFraction: 0.2);
IDataView trainDataView = trainTestData.TrainSet;
IDataView testDataView = trainTestData.TestSet;

//Define the model's training pipeline
var pipeline = mlContext.MulticlassClassification.Trainers
.ImageClassification(featureColumnName: "Image",
                     labelColumnName: "LabelAsKey",
                     validationSet: testDataView)
    .Append(mlContext.Transforms.Conversion.MapKeyToValue(outputColumnName: "PredictedLabel",
                                                          inputColumnName: "PredictedLabel"));

//Train
ITransformer trainedModel = pipeline.Fit(trainDataView);

//Evaluate
var predictionsDataView = trainedModel.Transform(testDataView);
var metrics = mlContext.MulticlassClassification.Evaluate(predictionsDataView, labelColumnName:"LabelAsKey", predictedLabelColumnName: "PredictedLabel");

Console.WriteLine($"Micro accuracy: {metrics.MicroAccuracy}");
Console.WriteLine($"Macro accuracy: {metrics.MacroAccuracy}");
           

Saver not created because there are no variables in the graph to restore
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\luisf\AppData\Local\Temp\23lj5dus.l2e\custom_retrained_model_based_on_resnet_v2_50_299.meta
Froze 2 variables. 
Converted 2 variables to const ops. 
Micro accuracy: 0,47161572052401746
Macro accuracy: 0,33066534914361


In [5]:
display(metrics);

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,TopKAccuracyForAllK,PerClassLogLoss,ConfusionMatrix
2.712796675191877,0.326177994991624,0.33066534914361,0.47161572052401746,0,0,<null>,"[ 0.36813301618152505, 4.547832096429718, 0, 2.566519547728572, 0, 4.0956394021024645, 1.239876431725156, 0, 0, 10.64014140736494, 0, 0, 0.9248550278154412, 0, 3.3035110456362453, 0, 10.547318905846522, 0, 0, 0 ... (155 more) ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.5, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0.3333333333333333, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], PerClassRecall: [ 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0.6666666666666666, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], Counts: [ [ 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 2, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (155 more) ] ... (155 more) ], NumberOfClasses: 175 }"


----

## After testing with the first model, it is possible to see that a low accuracy is achieved, possibly because the model was trained with about 120 label where the images were below 5, which means that such amount of images could result in overfitting, for example, one flower had only two images and both in a vase, when trying to predict this flower , if an image had no vase it would fail to predict this was the actual flower or present a lower accuracy than if the plant was in a vase. The macro accuracy being way lower than the micro accuracy, shows that there is some class imbalance, which proves that some labels are doing much better than others.

## The decision was to reduce the dataset and keep the labels where there is at least 5 images of a plant, reducing it from 181 labels to about 61 labels.

In [6]:
IEnumerable<ImageData> images = LoadImages(folder: initialReducedDataset, useFolderNameAsLabel: true);
IDataView fullImagesDataset = mlContext.Data.LoadFromEnumerable(images);
IDataView shuffledFullImageFilePathsDataset = mlContext.Data.ShuffleRows(fullImagesDataset);

IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.
    MapValueToKey(outputColumnName: "LabelAsKey", inputColumnName: "Label", keyOrdinality: KeyOrdinality.ByValue)
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: initialDataset,
        inputColumnName: "ImagePath"))
    .Fit(shuffledFullImageFilePathsDataset)
    .Transform(shuffledFullImageFilePathsDataset);

var trainTestData = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset, testFraction: 0.2);
IDataView trainDataView = trainTestData.TrainSet;
IDataView testDataView = trainTestData.TestSet;

var pipeline = mlContext.MulticlassClassification.Trainers
.ImageClassification(featureColumnName: "Image",
                     labelColumnName: "LabelAsKey",
                     validationSet: testDataView)
    .Append(mlContext.Transforms.Conversion.MapKeyToValue(outputColumnName: "PredictedLabel",
                                                          inputColumnName: "PredictedLabel"));

ITransformer trainedModelResnet = pipeline.Fit(trainDataView);

var predictionsDataView = trainedModelResnet.Transform(testDataView);
var metrics = mlContext.MulticlassClassification.Evaluate(predictionsDataView, labelColumnName:"LabelAsKey", predictedLabelColumnName: "PredictedLabel");

Console.WriteLine($"Micro accuracy: {metrics.MicroAccuracy}");
Console.WriteLine($"Macro accuracy: {metrics.MacroAccuracy}");

Saver not created because there are no variables in the graph to restore
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\luisf\AppData\Local\Temp\orccskky.nzz\custom_retrained_model_based_on_resnet_v2_50_299.meta
Froze 2 variables. 
Converted 2 variables to const ops. 
Micro accuracy: 0,5792349726775956
Macro accuracy: 0,5273421325051759


---- 

### Tuning hyperparameters to enhance the model

### Pre-trained Model choice fell upon MobilenetV2 since it reaches a compromise between performance and accuracy, in this case, it presented a higher accuracy than ResNet50 v2 as it can seen below: (ResNet was used so far, since it is the standard on ML.Net)
### MobilenetV2 is less heavy in size and also performs faster, about half the time for both training and predicting. 
###
### For the learning rate, it 0.01 seemed to have the best prediction, after both 0.001 and 0.0001 were tested.

In [7]:
IEnumerable<ImageData> images = LoadImages(folder: initialReducedDataset, useFolderNameAsLabel: true);
IDataView fullImagesDataset = mlContext.Data.LoadFromEnumerable(images);
IDataView shuffledFullImageFilePathsDataset = mlContext.Data.ShuffleRows(fullImagesDataset);

IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.
    MapValueToKey(outputColumnName: "LabelAsKey", inputColumnName: "Label", keyOrdinality: KeyOrdinality.ByValue)
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: initialDataset,
        inputColumnName: "ImagePath"))
    .Fit(shuffledFullImageFilePathsDataset)
    .Transform(shuffledFullImageFilePathsDataset);

var trainTestData = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset, testFraction: 0.2);
IDataView trainDataView = trainTestData.TrainSet;
IDataView testDataView = trainTestData.TestSet;

var options = new ImageClassificationTrainer.Options()
 {
    FeatureColumnName = "Image",
    LabelColumnName = "LabelAsKey",
    Arch = ImageClassificationTrainer.Architecture.MobilenetV2,
    Epoch = 50,
    BatchSize = 10,
    LearningRate = 0.01f,
    ValidationSet = testDataView
};

var pipeline = mlContext.MulticlassClassification.Trainers.ImageClassification(options)
        .Append(mlContext.Transforms.Conversion.MapKeyToValue(
           outputColumnName: "PredictedLabel",
           inputColumnName: "PredictedLabel"));

ITransformer trainedModelMobilenet = pipeline.Fit(trainDataView);

var predictionsDataView = trainedModelMobilenet.Transform(testDataView);
var metrics1 = mlContext.MulticlassClassification.Evaluate(predictionsDataView, labelColumnName:"LabelAsKey", predictedLabelColumnName: "PredictedLabel");

Saver not created because there are no variables in the graph to restore
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\luisf\AppData\Local\Temp\jr3imhai.jng\custom_retrained_model_based_on_mobilenet_v2.meta
Froze 2 variables. 
Converted 2 variables to const ops. 


In [8]:
metrics1

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,TopKAccuracyForAllK,PerClassLogLoss,ConfusionMatrix
1.1737705547602693,0.6632914222570694,0.648350500074638,0.7049180327868853,0,0,<null>,"[ 0.1608556668929925, 0.25651922065060995, 2.2576713171020333, 0, 1.8445085740779537, 6.6854746861317516, 0.7277736990958644, 1.384780834001203, 0, 0.332261426768962, 3.472820753735914, 0.2840572173333149, 0.5363352768772779, 0, 2.349795337708791, 0, 2.9026964386525718, 2.7655315673679914, 0.09544686259546195, 0 ... (38 more) ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 0.75, 0.75, 1, 0, 0, 0, 0.8333333333333334, 0.5, 0, 0.6923076923076923, 0, 1, 0.42857142857142855, 0, 0, 0, 1, 1, 0.3333333333333333, 0 ... (38 more) ], PerClassRecall: [ 1, 1, 0.5, 0, 0, 0, 1, 0.3333333333333333, 0, 0.9, 0, 1, 0.75, 0, 0, 0, 0.5, 0.3333333333333333, 1, 0 ... (38 more) ], Counts: [ [ 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 3, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 1, 0, 2, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 5, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 9, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 3, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0 ... (38 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (38 more) ] ... (38 more) ], NumberOfClasses: 58 }"


### The accuracy is still low, since the images are not that varied in some classes, so the next step was to process the data inside the dataset and multiply the images, such as rotating them and decreasing the quality to improve the accuracy when different images are evaluated. This step was performed in the project DataManipulation that contains the methods to perform this, consisting of flipping, rotating, mirroring and adjusting the quality of the image.

### The whole reason behind doing the data manipulation is two-fold. The first reason is to scale our dataset as collecting data is and always will be a hot topic for machine learning and the second, but nonetheless important, reason is that playing with the image in this sort of way allows the neural network to distill the truly useful points in the image - for instance the positionining of the rose petals should not matter nor the quality of the picture. This is much akin to feeding a machine learning algorithm a bunch of left facing cars that we can call car A and then feeding it any other car that's not car A and if it learned that the left facing characteristic is only for car A it will predict only it. This was something that we wanted to alleviate using manipulation - in a scenario where our data would be in the 7 digit numbers this would not be as needed.

### Then the new model is trained with the chosen pre-trained model but also with the default one (ResNetv2 50) to compare the difference between them. It is possible to see that the one using MobilenetV2 has a slightly better accuracy and a lower log loss.
### The data manipulation proved to be efficient, raising the accuracy from about 65% to around 99,5% on Mobilenet v2.

In [9]:
IEnumerable<ImageData> images = LoadImages(folder: finalDataset, useFolderNameAsLabel: true);
IDataView fullImagesDataset = mlContext.Data.LoadFromEnumerable(images);
IDataView shuffledFullImageFilePathsDataset = mlContext.Data.ShuffleRows(fullImagesDataset);

IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.
    MapValueToKey(outputColumnName: "LabelAsKey", inputColumnName: "Label", keyOrdinality: KeyOrdinality.ByValue)
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: finalDataset,
        inputColumnName: "ImagePath"))
    .Fit(shuffledFullImageFilePathsDataset)
    .Transform(shuffledFullImageFilePathsDataset);

var trainTestData = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset, testFraction: 0.2);
IDataView trainDataView = trainTestData.TrainSet;
IDataView testDataView = trainTestData.TestSet;

var options = new ImageClassificationTrainer.Options()
 {
    FeatureColumnName = "Image",
    LabelColumnName = "LabelAsKey",
    Arch = ImageClassificationTrainer.Architecture.MobilenetV2,
    Epoch = 50,       //100
    BatchSize = 10,
    LearningRate = 0.01f,
    ValidationSet = testDataView
};

var pipeline = mlContext.MulticlassClassification.Trainers.ImageClassification(options)
        .Append(mlContext.Transforms.Conversion.MapKeyToValue(
           outputColumnName: "PredictedLabel",
           inputColumnName: "PredictedLabel"));

ITransformer trainedModelMobilenetDataMan = pipeline.Fit(trainDataView);

var predictionsDataView = trainedModelMobilenetDataMan.Transform(testDataView);
var metrics2 = mlContext.MulticlassClassification.Evaluate(predictionsDataView, labelColumnName:"LabelAsKey", predictedLabelColumnName: "PredictedLabel");

Saver not created because there are no variables in the graph to restore
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\luisf\AppData\Local\Temp\4eexd5b3.0h5\custom_retrained_model_based_on_mobilenet_v2.meta
Froze 2 variables. 
Converted 2 variables to const ops. 


In [10]:
metrics2

LogLoss,LogLossReduction,MacroAccuracy,MicroAccuracy,TopKAccuracy,TopKPredictionCount,TopKAccuracyForAllK,PerClassLogLoss,ConfusionMatrix
0.027938287411030518,0.9925660305550863,0.9909150094713843,0.9913132345426674,0,0,<null>,"[ 0.017591218290972238, 0.005978731595138657, 0.013744333365584057, 0.1499154265946054, 0.004811479221092493, 0.07750929859822621, 0.009219002966385677, 0.13674425032859366, 0.03290198601511861, 0.0059070260931522135, 0.014082153718429986, 0.03696582765489416, 0.004920939503376701, 0.017055632573227564, 0.03521488771068378, 0.03414178015938873, 0.06183768922734698, 0.008068498794490314, 0.015629979979705135, 0.01835390657302592 ... (41 more) ]","{ Microsoft.ML.Data.ConfusionMatrix: PerClassPrecision: [ 1, 1, 1, 0.9819819819819819, 1, 1, 1, 0.9763779527559056, 0.9647577092511013, 1, 1, 1, 1, 0.9736842105263158, 1, 0.8048780487804879, 1, 1, 1, 1 ... (41 more) ], PerClassRecall: [ 1, 1, 1, 0.9316239316239316, 1, 0.9791666666666666, 1, 0.9465648854961832, 0.9909502262443439, 1, 1, 1, 1, 0.9946236559139785, 1, 1, 1, 1, 1, 1 ... (41 more) ], Counts: [ [ 109, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 33, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 104, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 109, 0, 0, 0, 0, 8, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 31, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 47, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 43, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 124, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 2, 0, 0, 0, 0, 219, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 38, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 352, 0, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 56, 0, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 25, 0, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 185, 0, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 39, 0, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 33, 0, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 40, 0, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 52, 0, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 113, 0 ... (41 more) ], [ 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 65 ... (41 more) ] ... (41 more) ], NumberOfClasses: 61 }"


In [12]:
IEnumerable<ImageData> images = LoadImages(folder: finalDataset, useFolderNameAsLabel: true);
IDataView fullImagesDataset = mlContext.Data.LoadFromEnumerable(images);
IDataView shuffledFullImageFilePathsDataset = mlContext.Data.ShuffleRows(fullImagesDataset);

IDataView shuffledFullImagesDataset = mlContext.Transforms.Conversion.
    MapValueToKey(outputColumnName: "LabelAsKey", inputColumnName: "Label", keyOrdinality: KeyOrdinality.ByValue)
    .Append(mlContext.Transforms.LoadRawImageBytes(
        outputColumnName: "Image",
        imageFolder: finalDataset,
        inputColumnName: "ImagePath"))
    .Fit(shuffledFullImageFilePathsDataset)
    .Transform(shuffledFullImageFilePathsDataset);

var trainTestData = mlContext.Data.TrainTestSplit(shuffledFullImagesDataset, testFraction: 0.2);
IDataView trainDataView = trainTestData.TrainSet;
IDataView testDataView = trainTestData.TestSet;

var pipeline = mlContext.MulticlassClassification.Trainers
.ImageClassification(featureColumnName: "Image",
                     labelColumnName: "LabelAsKey",
                     validationSet: testDataView)
    .Append(mlContext.Transforms.Conversion.MapKeyToValue(outputColumnName: "PredictedLabel",
                                                          inputColumnName: "PredictedLabel"));

ITransformer trainedModelResnetDataMan = pipeline.Fit(trainDataView);

var predictionsDataView = trainedModelResnetDataMan.Transform(testDataView);
var metrics3 = mlContext.MulticlassClassification.Evaluate(predictionsDataView, labelColumnName:"LabelAsKey", predictedLabelColumnName: "PredictedLabel");

Saver not created because there are no variables in the graph to restore
Saver not created because there are no variables in the graph to restore
Restoring parameters from C:\Users\luisf\AppData\Local\Temp\2sbiytvf.red\custom_retrained_model_based_on_resnet_v2_50_299.meta
Froze 2 variables. 
Converted 2 variables to const ops. 


In [13]:
Console.WriteLine($"Micro accuracy: {metrics3.MicroAccuracy}");
Console.WriteLine($"Macro accuracy: {metrics3.MacroAccuracy}");
Console.WriteLine($"Log loss: {metrics3.LogLoss}");
Console.WriteLine($"Log loss reduction: {metrics3.LogLossReduction}");

Micro accuracy: 0,9921648782149549
Macro accuracy: 0,9913303593965694
Log loss: 0,039259256162941465
Log loss reduction: 0,9895473451372309


### To conclude, the team collected some images from outside the dataset to test and to simulate the user input, for each of the trained models.

In [51]:
string rose = @"C:\Users\luisf\Downloads\TestImages\Image\rose";
TrySinglePrediction(rose,mlContext,trainedModelResnet);
TrySinglePrediction(rose,mlContext,trainedModelMobilenet);
TrySinglePrediction(rose,mlContext,trainedModelMobilenetDataMan);
TrySinglePrediction(rose,mlContext,trainedModelResnetDataMan);
System.Console.WriteLine("---------------- Expected: Rose ------------");

string chrysanthemum = @"C:\Users\luisf\Downloads\TestImages\Image\chrysanthemum";
TrySinglePrediction(chrysanthemum,mlContext,trainedModelResnet);
TrySinglePrediction(chrysanthemum,mlContext,trainedModelMobilenet);
TrySinglePrediction(chrysanthemum,mlContext,trainedModelMobilenetDataMan);
TrySinglePrediction(chrysanthemum,mlContext,trainedModelResnetDataMan);
System.Console.WriteLine("---------------- Expected: Chrysanthemum ------------");

string grape = @"C:\Users\luisf\Downloads\TestImages\Image\grape";
TrySinglePrediction(grape,mlContext,trainedModelResnet);
TrySinglePrediction(grape,mlContext,trainedModelMobilenet);
TrySinglePrediction(grape,mlContext,trainedModelMobilenetDataMan);
TrySinglePrediction(grape,mlContext,trainedModelResnetDataMan);
System.Console.WriteLine("---------------- Expected: Grape ------------");

string cactus = @"C:\Users\luisf\Downloads\TestImages\Image\cactus";
TrySinglePrediction(cactus,mlContext,trainedModelResnet);
TrySinglePrediction(cactus,mlContext,trainedModelMobilenet);
TrySinglePrediction(cactus,mlContext,trainedModelMobilenetDataMan);
TrySinglePrediction(cactus,mlContext,trainedModelResnetDataMan);
System.Console.WriteLine("---------------- Expected: Cactus ------------");


string doubleFlower = @"C:\Users\luisf\Downloads\TestImages\Image\double";
TrySinglePrediction(doubleFlower,mlContext,trainedModelResnet);
TrySinglePrediction(doubleFlower,mlContext,trainedModelMobilenet);
TrySinglePrediction(doubleFlower,mlContext,trainedModelMobilenetDataMan);
TrySinglePrediction(doubleFlower,mlContext,trainedModelResnetDataMan);
System.Console.WriteLine("---------------- Expected: Rose or Hortensia ------------");

string yucca = @"C:\Users\luisf\Downloads\TestImages\Image\yucca";
TrySinglePrediction(yucca,mlContext,trainedModelResnet);
TrySinglePrediction(yucca,mlContext,trainedModelMobilenet);
TrySinglePrediction(yucca,mlContext,trainedModelMobilenetDataMan);
TrySinglePrediction(yucca,mlContext,trainedModelResnetDataMan);
System.Console.WriteLine("---------------- Expected: Yucca ------------");


Image Filename : [pexels-yaroslava-borz-10049901.jpg], Scores : [0,47827196], Predicted Label : chrysanthemum
Image Filename : [pexels-yaroslava-borz-10049901.jpg], Scores : [0,74887437], Predicted Label : rose
Image Filename : [pexels-yaroslava-borz-10049901.jpg], Scores : [0,9900767], Predicted Label : rose
Image Filename : [pexels-yaroslava-borz-10049901.jpg], Scores : [0,99600214], Predicted Label : chrysanthemum
---------------- Expected: Rose ------------
Image Filename : [flowers-10201_640.jpg], Scores : [0,80124414], Predicted Label : chrysanthemum
Image Filename : [flowers-10201_640.jpg], Scores : [0,9545941], Predicted Label : chrysanthemum
Image Filename : [flowers-10201_640.jpg], Scores : [0,99988496], Predicted Label : chrysanthemum
Image Filename : [flowers-10201_640.jpg], Scores : [0,98275304], Predicted Label : chrysanthemum
---------------- Expected: Chrysanthemum ------------
Image Filename : [pexels-sharath-g-4186544.jpg], Scores : [0,8889381], Predicted Label : grap

### It can be induced from this results that different plants cause misclassification at times, depending on the model. The models using ResNet, since it is a deeper network, recognize plants by more attributes, while Mobilenet can misclassify a plant if there is some similarities, like the same color is presented, even if the plant has a different shape.
### The results also show that the data manipulation did improve the accuracy of the model in general, and also reduced the imbalance between classes, if we take by example the Cactus, the initial models tend to take Rose and Grape since these are the labels with the biggest amount of data, and after the data manipulation, even if the prediction is wrong, it takes in account flowers with less data, which have more similarities with the flower of the cactus than the rose or grape.